In [ ]:
%pip install numpy pandas pickle
%pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import pickle


class ConvolutionLayer:
    def __init__(self, shape, n_filters, kernel_size, stride=1, padding=0):
        self.shape = shape
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
    
    def forward(self, X):
        n_samples = X.shape[0]
        n_input = X.shape[1]
        n_channels = X.shape[2]
